In [14]:
import pandas as pd

In [15]:
# df = pd.read_csv('./data/rss_slices/3day_window_20250530T1200.csv')


# # Save as JSONL
# output_path = './data/rss_slices/3day_window_20250530T1200.jsonl'
# df.to_json(output_path, orient='records', lines=True, force_ascii=False)

In [16]:
import pandas as pd
import numpy as np
import os


def split_topic_groups(df, min_rows=5, max_rows=25):
    result_frames = []

    for topic, group in df.groupby('Topic'):
        N = len(group)

        # Determine number of groups
        num_groups = max(1, int(np.ceil(N / max_rows)))
        split_size = int(np.ceil(N / num_groups))

        # print(topic, N, num_groups, split_size)

        # Safety check: if split_size too small, fallback to one group
        if split_size < min_rows:
            group = group.copy()
            group_id = f"01"
            group['GroupID'] = group_id
            result_frames.append(group)
        else:
            splits = np.array_split(group, num_groups)
            for i, split in enumerate(splits, start=1):
                split = split.copy()
                group_id = f"{i:02d}"
                split['GroupID'] = group_id
                result_frames.append(split)

    grouped_df = pd.concat(result_frames, ignore_index=True)
    return grouped_df


import os
import re

def sanitize_topic(topic):
    # Replace spaces and special characters with underscores
    return re.sub(r'[^\w\-]', '_', topic.strip().replace(' ', '_'))

def save_digest_files(df_grouped, output_dir, filename_prefix):
    os.makedirs(output_dir, exist_ok=True)
    group_metadata = []

    for (topic, group_id), group in df_grouped.groupby(['Topic', 'GroupID']):
        safe_topic = sanitize_topic(topic)
        filename = f"headlines_{filename_prefix}_{safe_topic}_{group_id}.md"
        filepath = os.path.join(output_dir, filename)

        # Compose markdown with headlines
        content = f"# {topic} (Grupo {group_id})\n\n"
        for idx, row in group.iterrows():
            article_id = row['article_id']
            title = row['Title']
            published = row.get('Published', '')
            source = row.get('Source', '')

            line = f"ID: {article_id} - Title: {title}"
            if published:
                try:
                    published_dt = pd.to_datetime(published)
                    line += f" _(Publicado: {published_dt.strftime('%Y-%m-%d %Hhs')})_"
                except Exception:
                    pass
            if source:
                line += f" — _Fuente: {source}_"
            content += line + "\n"

        # Save the markdown file
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(content)

        # Save metadata for downstream
        group_metadata.append({
            'group_id': group_id,
            'topic': topic,
            'filename': filename,
            'num_articles': len(group)
        })

    return group_metadata


# Example usage:
# df = pd.read_csv('path/to/csv')
# df_grouped, group_metadata = split_topic_groups(df)
# save_digest_files(df_grouped, './output_digests/')


In [17]:
import warnings

warnings.filterwarnings('ignore')

In [18]:
import os
import glob
import pandas as pd

input_dir = './data/rss_slices/'
output_dir = './data/output_digests/'

# Collect all CSVs in the input directory
csv_files = glob.glob(os.path.join(input_dir, '*.csv'))

for csv_file in csv_files[:10]: ### fast
    df = pd.read_csv(csv_file)
    if 'article_id' not in df.columns: 
        df['article_id'] = range(len(df))
    df = df.sort_values('Published')
    df_grouped = split_topic_groups(df)
    filename_prefix = os.path.splitext(os.path.basename(csv_file))[0]  # e.g. '2day_window_20250528T1200'
    group_metadata = save_digest_files(df_grouped, output_dir, filename_prefix)   ## SAVED
    print(f"✅ Digest files created for: {filename_prefix}")



# data/rss_slices/2day_window_20250530T1200.csv [article_id, [Title, Source]] -- []

✅ Digest files created for: 4h_window_20250611T0400
✅ Digest files created for: 4h_window_20250531T2000
✅ Digest files created for: 8h_window_20250606T1600
✅ Digest files created for: 4h_window_20250530T1200
✅ Digest files created for: 4h_window_20250606T1200
✅ Digest files created for: 2day_window_20250604T1200
✅ Digest files created for: 2day_window_20250607T1200
✅ Digest files created for: 4h_window_20250603T0400
✅ Digest files created for: 4h_window_20250605T1600
✅ Digest files created for: 4h_window_20250602T1600


In [47]:
import glob
import os
import json

def parse_filename(filename):
    """
    Parses a filename like:
    headlines_2day_window_20250531T1200_Actividad_y_Empleo_01.md
    """
    base = os.path.basename(filename)
    name, _ = os.path.splitext(base)
    parts = name.split('_')
    
    # Defensive parsing — ensure correct length
    if len(parts) < 6:
        raise ValueError(f"Unexpected filename format: {filename}")
    
    label = parts[0]
    window_type = parts[1]
    window_label = parts[2]  # usually 'window'
    datetime_str = parts[3]
    topic = '_'.join(parts[4:-1])  # handle multi-word topics
    group_number = parts[-1]
    
    return {
        "headlines_file": filename,
        "label": label,
        "window_type": window_type,
        "datetime": datetime_str,
        "topic": topic,
        "group_number": group_number
    }

def create_digest_jsonl(input_dir, date_filter, output_file):
    pattern = os.path.join(input_dir, f"headlines_*_{date_filter}*.md")
    files = glob.glob(pattern)
    print(f"Found {len(files)} files matching {date_filter}")
    
    output_lines = []
    
    for i, file in enumerate(files):
        metadata = parse_filename(file)
        with open(file, 'r', encoding='utf-8') as f:
            content = f.read()
        metadata['id_digest'] = i
        metadata["content"] = content
        
        # Add to output lines
        output_lines.append(json.dumps(metadata, ensure_ascii=False))
    
    with open(output_file, 'w', encoding='utf-8') as out_f:
        for line in output_lines:
            out_f.write(line + '\n')
    
    print(f"✅ Saved {len(output_lines)} records to {output_file}")

# Example usage:
# create_digest_jsonl('./data/output_digests/', '20250606T20', './data/digest_jsonls/2day_window_20250606T20.jsonl')


create_digest_jsonl('./data/output_digests/', '20250530T12', './data/digest_jsonls/20250530T12.jsonl')

Found 26 files matching 20250530T12
✅ Saved 26 records to ./data/digest_jsonls/20250530T12.jsonl


In [21]:
# ✅ Saved 26 records to ./data/digest_jsonls/2day_window_20250530T12.jsonl


In [36]:
import os
import json
import pandas as pd
from glob import glob

# Ruta al archivo digest jsonl para una hora específica
digest_jsonl_path = '/home/matias/Documents/media_monitor/data/digest_jsonls/20250530T12.jsonl'
window_id = '20250530T12'

# Cargar todos los objetos JSONL
records = []
with open(digest_jsonl_path, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            records.append(json.loads(line))
        except json.JSONDecodeError:
            continue

# Explode records por ID dentro del campo 'content'
expanded = []
for rec in records:
    digest_file = os.path.basename(rec['headlines_file'])
    prefix = digest_file.replace('headlines_', '').replace('.md', '')
    lines = rec['content'].splitlines()
    for line in lines:
        if line.strip().startswith("ID: "):
            parts = line.split(" - Title: ")
            if len(parts) < 2:
                continue
            try:
                article_id = int(parts[0].replace("ID:", "").strip())
                title_part = parts[1].split("_(Publicado:")[0].strip()
                source_part = parts[1].split("_— _Fuente: ")[-1].replace("_", "").strip()
                expanded.append({
                    "filename": f"{rec['window_type']}_window_{window_id}00",
                    "article_id": article_id,
                    "Title": title_part,
                    "Source": source_part,
                })
            except Exception:
                continue

df_expanded = pd.DataFrame(expanded)
df_expanded = df_expanded.drop_duplicates(subset=["article_id", "Title", "Source"])

In [37]:
df_expanded

,filename,article_id,Title,Source
0,2day_window_20250530T1200,149,Tribunal de EEUU anula la mayoría de los aranc...,Tribunal de EEUU anula la mayoría de los aranc...
1,2day_window_20250530T1200,163,Cortes de carreteras y pintadas en el arranque...,Cortes de carreteras y pintadas en el arranque...
2,2day_window_20250530T1200,143,Los tribunales bloquean los aranceles de Trump...,Los tribunales bloquean los aranceles de Trump...
3,2day_window_20250530T1200,150,Hassett dice que EE.UU. tiene otras opciones p...,Hassett dice que EE.UU. tiene otras opciones p...
4,2day_window_20250530T1200,166,La presión aumenta sobre un Trump reticente a ...,La presión aumenta sobre un Trump reticente a ...
...,...,...,...,...
373,2day_window_20250530T1200,57,"Con una tasa más alta que lo esperado, el Gobi...","Con una tasa más alta que lo esperado, el Gobi..."
374,2day_window_20250530T1200,65,Argentina confirma apetito por Bonte 2030 y su...,Argentina confirma apetito por Bonte 2030 y su...
375,2day_window_20250530T1200,64,Caputo logró u$s 1000 millones con el Bonte al...,Caputo logró u$s 1000 millones con el Bonte al...
376,2day_window_20250530T1200,87,El Tesoro consiguió otros u$s 1000 millones | ...,El Tesoro consiguió otros u$s 1000 millones | ...


In [ ]:
# EJECUCION DE PROMPTFLOW

# (aios-env) matias@matias-ThinkPad-T470-W10DG:~/Documents/media_monitor/flow$ python -m promptflow._cli.pf run create --flow . --data ./20250530T12.jsonl 

In [ ]:
# output_file = "/home/matias/.promptflow/.runs/flow_variant_0_20250611_031316_394094/flow_outputs/output.jsonl"
output_file = "/home/matias/.promptflow/.runs/flow_variant_0_20250611_051259_983174/flow_outputs/output.jsonl"
    # "output": "/home/matias/.promptflow/.runs/flow_variant_0_20250611_050531_220018/flow_outputs"
    # "output": "/home/matias/.promptflow/.runs/flow_variant_0_20250611_051259_983174/flow_outputs"

df = pd.read_json(output_file, lines = True)
df

,line_number,id_digest,window_type,topic,group_number,clustered_agenda_table,seed_ideas
0,0,24,2day,Personajes_Políticos_y_Económicos,1,{'clustered_agenda_table': [{'topic': 'Controv...,"{'seed_ideas': [{'idea_id': 'CK01', 'topic': '..."
1,1,25,2day,Deuda_y_Financiamiento,1,{'clustered_agenda_table': [{'topic': 'Aumento...,"{'seed_ideas': [{'idea_id': 'AI01', 'topic': '..."


In [39]:
import pandas as pd
import json

# Archivo JSONL de PromptFlow (ruta hardcodeada para el ejemplo)
output_file = "/home/matias/.promptflow/.runs/flow_variant_0_20250611_184234_952229/flow_outputs/output.jsonl"
    # "output": "/home/matias/.promptflow/.runs/flow_variant_0_20250611_184234_952229/flow_outputs"
# Cargar el archivo JSONL
df = pd.read_json(output_file, lines=True)

# Expandir cada cluster con metadata básica
all_article_rows = []

for i, row in df.iterrows():
    digest_id = row["id_digest"]
    window_type = row["window_type"]
    group_number = row["group_number"]
    digest_timestamp = "20250530T1200"  # FIXME: reconstruir esto automáticamente si posible

    clusters = row['clustered_agenda_table']['clustered_agenda_table']
    for cluster in clusters:
        topic = cluster["topic"]
        article_ids = cluster["article_ids"]
        titles = cluster["deduplicated_titles"]
        for aid, title in zip(article_ids, titles):
            all_article_rows.append({
                "filename": f"{window_type}_window_{digest_timestamp}",
                "article_id": aid,
                "Title": title,
                "Topic": topic,
                "group_number": group_number,
                "id_digest": digest_id
            })

# Crear DataFrame final con un artículo por fila
df_articles = pd.DataFrame(all_article_rows)



,filename,article_id,Title,Topic,group_number,id_digest
0,2day_window_20250530T1200,149,Tribunal de EEUU anula la mayoría de los aranc...,Bloqueo judicial a los aranceles de Trump,2,0
1,2day_window_20250530T1200,143,Los tribunales bloquean los aranceles de Trump...,Bloqueo judicial a los aranceles de Trump,2,0
2,2day_window_20250530T1200,150,Hassett dice que EE.UU. tiene otras opciones p...,Bloqueo judicial a los aranceles de Trump,2,0
3,2day_window_20250530T1200,154,Los mercados reaccionan con cautela ante el bl...,Bloqueo judicial a los aranceles de Trump,2,0
4,2day_window_20250530T1200,161,“Ganamos”: el mensaje de Kathy Hochul tras el ...,Bloqueo judicial a los aranceles de Trump,2,0
...,...,...,...,...,...,...
335,2day_window_20250530T1200,59,Licitación exitosa: el Gobierno consiguió USD ...,Emisión y licitación de bonos para financiamiento,1,25
336,2day_window_20250530T1200,57,"Con una tasa más alta que lo esperado, el Gobi...",Emisión y licitación de bonos para financiamiento,1,25
337,2day_window_20250530T1200,65,Argentina confirma apetito por Bonte 2030 y su...,Emisión y licitación de bonos para financiamiento,1,25
338,2day_window_20250530T1200,64,Caputo logró u$s 1000 millones con el Bonte al...,Emisión y licitación de bonos para financiamiento,1,25


In [41]:
master_ref = pd.read_csv('./data/master_ref.csv')

In [45]:
df_articles.merge(master_ref, left_on = ['filename', 'article_id'], right_on = ['digest_file' ,'article_id'])

,filename,article_id,Title_x,Topic,group_number,id_digest,Unnamed: 0,digest_file,index_id,Source,Title_y,Published
0,2day_window_20250530T1200,149,Tribunal de EEUU anula la mayoría de los aranc...,Bloqueo judicial a los aranceles de Trump,2,0,11002,2day_window_20250530T1200,f3d80db9,Clarin.com,El dólar oficial no afloja: rozó los $ 1.200 y...,2025-05-29 15:36:40+00:00
1,2day_window_20250530T1200,143,Los tribunales bloquean los aranceles de Trump...,Bloqueo judicial a los aranceles de Trump,2,0,10996,2day_window_20250530T1200,630c6d7c,elDiarioAR.com,Punto por punto: cuál es la estrategia de Capu...,2025-05-29 15:06:53+00:00
2,2day_window_20250530T1200,150,Hassett dice que EE.UU. tiene otras opciones p...,Bloqueo judicial a los aranceles de Trump,2,0,11003,2day_window_20250530T1200,99f56977,DataClave,Estalló Dalma Maradona por una foto de Diego c...,2025-05-29 15:42:29+00:00
3,2day_window_20250530T1200,154,Los mercados reaccionan con cautela ante el bl...,Bloqueo judicial a los aranceles de Trump,2,0,11007,2day_window_20250530T1200,c5fa47c8,LV12,Paritarias: Amado habló sobre la convocatoria ...,2025-05-29 15:57:00+00:00
4,2day_window_20250530T1200,161,“Ganamos”: el mensaje de Kathy Hochul tras el ...,Bloqueo judicial a los aranceles de Trump,2,0,11014,2day_window_20250530T1200,6d04b3f4,Perfil,El Gobierno festejó el Bonte y analiza su reed...,2025-05-29 16:59:59+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
333,2day_window_20250530T1200,59,Licitación exitosa: el Gobierno consiguió USD ...,Emisión y licitación de bonos para financiamiento,1,25,10912,2day_window_20250530T1200,f349e116,Infobae,"Se hundió, de nuevo, el proyecto para reducir ...",2025-05-28 20:58:00+00:00
334,2day_window_20250530T1200,57,"Con una tasa más alta que lo esperado, el Gobi...",Emisión y licitación de bonos para financiamiento,1,25,10910,2day_window_20250530T1200,112c1336,La Nación,Elisa Carrió apuntó contra Luis Caputo por la ...,2025-05-28 20:46:00+00:00
335,2day_window_20250530T1200,65,Argentina confirma apetito por Bonte 2030 y su...,Emisión y licitación de bonos para financiamiento,1,25,10918,2day_window_20250530T1200,10f84540,El Economista: Últimas noticias económicas y f...,"Carrió amenaza a Caputo por el Garrahan: ""Vas ...",2025-05-28 21:26:00+00:00
336,2day_window_20250530T1200,64,Caputo logró u$s 1000 millones con el Bonte al...,Emisión y licitación de bonos para financiamiento,1,25,10917,2day_window_20250530T1200,b64a900f,iProfesional,Luis Caputo consiguió u$s1.000 millones de inv...,2025-05-28 21:22:00+00:00


In [137]:
import pandas as pd

# Leer el archivo JSONL
# output_file = "/home/matias/.promptflow/.runs/flow_variant_0_20250611_040706_348318/flow_outputs/output.jsonl"
df = pd.read_json(output_file, lines=True)

# Inicializar una lista para ir juntando todas las filas de todos los batches
all_rows = []

# Iterar sobre cada línea
for i, row in df.iterrows():
    clusters = row['seed_ideas']['seed_ideas']
    for cluster in clusters:
        # Agregar contexto adicional si lo necesitas (ej: id de línea o id global)
        cluster['id_digest'] = row['id_digest']
        cluster['line_number'] = row['line_number']
        cluster['window_type'] = row['window_type']
        cluster['topic'] = row['topic']
        cluster['group_number'] = row['group_number']
        cluster['id_digest'] = row['id_digest']
        all_rows.append(cluster)

# Convertir a DataFrame final
final_df = pd.DataFrame(all_rows)

# Mostrar
display(final_df)


,idea_id,topic,idea_title,source_ids,draft_editorial_angle,key_data_points,potential_controversies,relevant_quotes,id_digest,line_number,window_type,group_number
0,CK01,Personajes_Políticos_y_Económicos,El Debate del Autoatentado: ¿Realidad o Estrat...,"[197, 220]",Explorar las distintas narrativas en torno al ...,[Declaraciones de la fiscal desestimando la te...,[La veracidad de las acusaciones de autoatenta...,['Es un disparate decir que fue un autoatentad...,24,0,2day,1
1,JM01,Personajes_Políticos_y_Económicos,Javier Milei: Reformas y Retos de un Gobierno ...,"[199, 232, 218, 230, 216, 225, 222, 177, 208, ...",Analizar las políticas implementadas por el go...,"[Desvinculación de 48,000 empleados estatales....","[Despidos masivos en el sector público., Regre...",['Estuvimos en manos de curanderos' - Responsa...,24,0,2day,1
2,AK01,Personajes_Políticos_y_Económicos,Axel Kicillof: Entre la Resistencia y la Campaña,"[233, 221, 228, 196]",Examinar cómo Axel Kicillof maneja las crítica...,"[Acusaciones de tener 'mentalidad soviética'.,...","[Tensiones dentro del kirchnerismo., Acusacion...",['El cristinismo dice que la tropa de Axel Kic...,24,0,2day,1
3,EC01,Personajes_Políticos_y_Económicos,Elisa Carrió vs. Luis Caputo: La Batalla por e...,"[186, 212]",Investigar el conflicto entre Elisa Carrió y L...,"[Carrió amenaza a Caputo con prisión., Crisis ...",[Responsabilidad en la gestión del Hospital Ga...,['Vas a terminar preso' - Elisa Carrió a Luis ...,24,0,2day,1
4,CE01,Personajes_Políticos_y_Económicos,Estrategias Económicas: Lecciones del Pasado y...,"[211, 181]",Discutir las advertencias económicas recientes...,[Plan de Federico Furiase para sumar dólares s...,[Efectividad de las estrategias económicas pro...,['Macri lo aprendió por las malas' - Alfonso P...,24,0,2day,1
5,RL01,Personajes_Políticos_y_Económicos,Ramón Lanús y la Política de Gestos: Milei vs....,[187],Explorar cómo las declaraciones de Ramón Lanús...,[Lanús no repudia la decisión de Milei de nega...,[Tensiones personales y políticas entre Milei ...,['Es su forma de ser' - Ramón Lanús sobre Milei.],24,0,2day,1
6,KM01,Personajes_Políticos_y_Económicos,Karina Milei y Patricia Bullrich: Una Alianza ...,[193],Analizar la potencial alianza política entre K...,[Karina Milei abre una ventana a Bullrich para...,"[Impacto de la alianza en la política local., ...",['Le abre una ventana a Patricia Bullrich' - s...,24,0,2day,1
7,AI01,Deuda_y_Financiamiento,Bancos argentinos ofrecen tasas históricas en ...,"[80, 62, 74, 76, 84]",Explorar cómo los bancos en Argentina están of...,[Tasas de interés en dólares superiores al 5% ...,[Impacto en la economía local al atraer más dó...,['Los bancos en Argentina están ofreciendo tas...,25,1,2day,1
8,AI02,Deuda_y_Financiamiento,Argentina asegura financiamiento internacional...,"[69, 58, 82, 81, 56, 85, 59, 57, 87]",Analizar el éxito del gobierno argentino en as...,"[Emisión de deuda por USD 1.000 millones., Fue...",[Dependencia de financiamiento externo y sus r...,['El nuevo bono es visto como una medida clave...,25,1,2day,1
9,AI03,Deuda_y_Financiamiento,Bonte 2030: ¿Una oportunidad de inversión o un...,"[53, 54, 65, 64]","Evaluar si el Bonte 2030, con un rendimiento m...",[Rendimiento del Bonte 2030 menor al 30% anual...,[Riesgo de inflación y devaluación afectando e...,['El Bonte 2030 es una apuesta a largo plazo e...,25,1,2day,1


In [88]:
output_file

'/home/matias/.promptflow/.runs/flow_variant_0_20250611_031316_394094/flow_outputs/output.jsonl'

In [145]:
links = pd.read_csv('/home/matias/Documents/media_monitor/data/rss_slices/2day_window_20250530T1200.csv')
if 'article_id' not in links.columns: 
    links['article_id'] = range(len(links))
links = links.sort_values('Published')


links_slice = links.loc[links['article_id'].isin(final_df['source_ids'][0])]

L1 = links_slice['Link'].values[0]
L1

'https://news.google.com/rss/articles/CBMi4wFBVV95cUxPM2tQd3djc09PbEFBMU5fazA1Ujd6NGhSaE5aLTFxaHJZYVFoZUlsRTlqZThuNXBuZ0JhaWJlTFY2X1ExUTJ2azN1bW5oc3NDZDRPb3hNcW9EVEw0T0VncTh5dU1wRGhkZVYybWpXd04xLXRUeGVXOHgyd3oxNkNlOHV5OU9iUnU5MW52QnRZQ2J6ek5BMlFwV3dIcEZkWGlrOXZ4SUlnNF8tZUwyS1FodjFObXpNMkRKRjBaX1hCYjlJTXkwb1pISElJVHVoNENKcnJESFBjUllBcmtyQjdJZnZtQQ?oc=5'